In [11]:
import Pkg; 

if split(pwd(),"/")[end] == "prototyping"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end

using MorphoMolNotebooks
using MorphoMol
using JLD2
using GLMakie
using GeometryBasics

In [12]:
@load "../../Data/hpc_out/rwm_wip_2_6r7m/13_rwm_wip_2_6r7m.jld2" input output

2-element Vector{Symbol}:
 :input
 :output

In [35]:
n = length(output["Es"])
selection = [i for i in n-25:n];

In [39]:
realizations = [[e for e in eachcol(hcat(MorphoMol.Utilities.get_matrix_realization(state, input["template_centers"])...))] for state in output["states"][selection]]
realizations = [hcat(realizations[i]...) for i in 1:length(realizations)]
Es = output["Es"][selection]
pf = MorphoMol.Energies.get_prefactors(input["rs"], input["η"])
Vs = pf[1] .* output["Vs"][selection]
As = pf[2] .* output["As"][selection]
Cs = pf[3] .* output["Cs"][selection]
Xs = pf[4] .* output["Xs"][selection]
OLs = input["overlap_slope"] .* output["OLs"][selection]
mol_type = input["mol_type"]    
exp_template_centers = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["template_centers"]
exp_state = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["state"]
thetas = [MorphoMol.Utilities.average_offset_distance(exp_template_centers, input["template_centers"], exp_state, state) for state in output["states"][selection]]
xs = [i for i in 1:length(Es)];
zero_ps = output["P0s"][selection]
one_ps = output["P1s"][selection]
pw = input["persistence_weights"]
tps = zero_ps .* pw[1] .+ one_ps .* pw[2];

In [47]:
barycentric_subdivisions = Vector{Vector{Point3f}}([])
ifils = Vector{Vector{Tuple{Vector{Int}, Float32}}}([])

for i in 1:length(selection)
    points = [e for e in eachcol(realizations[i])]
    mc_tet = MorphoMol.Energies.get_multichromatic_tetrahedra(points, 1206)
    bcs, ifil = MorphoMol.Energies.get_barycentric_subdivision_and_filtration(points, mc_tet)
    push!(barycentric_subdivisions, bcs)
    push!(ifils, ifil)
end

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26


In [49]:
max_v = sqrt(maximum([v for ifil in ifils for (_, v) in ifil]))
min_v = sqrt(minimum([v for ifil in ifils for (_, v) in ifil]))

xs = [i for i in 1:length(Es)];

f = Figure(fontsize = 12)
title_fs = 15
ms = 5
cm = :Dark2_8
cr = (1,8)

#Slider
sl_i = Slider(f[6, 1:6], range = 1:length(xs), startvalue = 1)
x = sl_i.value

# Interface
ig = GridLayout(f[1:3, 4:6])
Label(ig[0, 1:2], text = "Interface", fontsize = title_fs)
i_sc = LScene(ig[1:2, 1:2], show_axis=false)

faces = [[TriangleFace(e[1]) for e in filtration if length(e[1]) == 3] for filtration in ifils]
meshes = [GeometryBasics.Mesh(bcs, fs) for (bcs, fs) in zip(barycentric_subdivisions, faces)]
colors = [[e[2] for e in filtration if length(e[1]) == 1] for filtration in ifils]

l_ms, l_cs = @lift(meshes[$x]), @lift(colors[$x])
mesh!(i_sc, l_ms, color = l_cs, colorrange = (min_v, max_v), colormap = :magma)


# Configuration
cg = GridLayout(f[1:2, 1:3])
Label(cg[0, 1:2], text = "Configuration", fontsize = title_fs)
conf_sc = LScene(cg[1:2, 1:2], show_axis=false)
n_atoms = size(realizations[1])[2]
n_mol = 2
points = @lift([Point3f(realizations[$x][1,i], realizations[$x][2,i], realizations[$x][3,i]) for i in 1:n_atoms])
conf_colors = vcat([[j for _ in 1:1206] for j in 1:n_mol]...)
conf_ms = 10
scatter!(conf_sc, points, markersize = conf_ms, color = conf_colors, colormap = :pastel)
mesh!(conf_sc, l_ms, color = l_cs, colorrange = (min_v, max_v), colormap = :magma)

# # Persistence
# cm = :Spectral_4
# cr = (1, 4)

# pg = GridLayout(f[1:2, 3:4])
# Label(pg[0, 1:3], text = "Persistence (not included in Energy)", fontsize = title_fs)
# zero_p_ax = Axis(pg[2, 1], title = "$(pw[1]) b_0")
# one_p_ax = Axis(pg[1, 2], title = "$(pw[2]) b_1")
# tp_ax = Axis(pg[1, 1], title = L"Σ")

# tp_mark = @lift(Point2f($x, $@lift(tps[$x])))

# zero_p_mark = @lift(Point2f($x, $@lift(zero_ps[$x])))
# one_p_mark = @lift(Point2f($x, $@lift(one_ps[$x])))
# scatter!(zero_p_ax, xs, zero_ps, color = 1, colormap = cm, colorrange = cr, markersize = ms)
# scatter!(zero_p_ax, zero_p_mark, color=:black, markersize = 10)

# scatter!(one_p_ax, xs, one_ps, color = 2, colormap = cm, colorrange = cr, markersize = ms)
# scatter!(one_p_ax, one_p_mark, color=:black, markersize = 10)

#Energy and Theta
eg = GridLayout(f[4, 4:6])
Label(eg[0, 1:2], text = "Energy", fontsize = title_fs)
E_ax = Axis(eg[1:2, 1], title = L"F_{sol}")
theta_ax = Axis(eg[1:2, 2], title = L"\Theta")

theta_mark = @lift(Point2f($x, $@lift(thetas[$x])))
E_mark = @lift(Point2f($x, $@lift(Es[$x])))

scatter!(theta_ax, xs, thetas, color = :magenta, markersize = ms)
scatter!(theta_ax, theta_mark, color=:black, markersize = 10)

scatter!(E_ax, xs, Es, color = :blue, markersize = ms)
scatter!(E_ax, E_mark, color=:black, markersize = 10)

#Measures    
pf = MorphoMol.Energies.get_prefactors(1.4, 0.3665)
Vs = pf[1] .* Vs
As = pf[2] .* As
Cs = pf[3] .* Cs
Xs = pf[4] .* Xs

mg = GridLayout(f[5, 1:5])
Label(mg[0, 1:5], text = "Geometric Measures", fontsize = title_fs)
cm = :Set2_5
cr = (1, 5)

Vs_ax = Axis(mg[1, 1], title = L"pV")
As_ax = Axis(mg[1, 2], title = L"\sigma A")
Cs_ax = Axis(mg[1, 3], title = L"\kappa C")
Xs_ax = Axis(mg[1, 4], title = L"\overline{\kappa} X")
OLs_ax = Axis(mg[1, 5], title = L"OLs")

Vs_mark = @lift(Point2f($x, $@lift(Vs[$x])))
As_mark = @lift(Point2f($x, $@lift(As[$x])))
Cs_mark = @lift(Point2f($x, $@lift(Cs[$x])))
Xs_mark = @lift(Point2f($x, $@lift(Xs[$x])))
OLs_mark = @lift(Point2f($x, $@lift(OLs[$x])))

scatter!(Vs_ax, xs, Vs, color = 1, colormap = cm, colorrange = cr, markersize = ms)
scatter!(Vs_ax, Vs_mark, color=:black, markersize = 10)

scatter!(As_ax, xs, As, color = 2, colormap = cm, colorrange = cr, markersize = ms)
scatter!(As_ax, As_mark, color=:black, markersize = 10)

scatter!(Cs_ax, xs, Cs, color = 3, colormap = cm, colorrange = cr, markersize = ms)
scatter!(Cs_ax, Cs_mark, color=:black, markersize = 10)

scatter!(Xs_ax, xs, Xs, color = 4, colormap = cm, colorrange = cr, markersize = ms)
scatter!(Xs_ax, Xs_mark, color=:black, markersize = 10)

scatter!(OLs_ax, xs, OLs, color = 5, colormap = cm, colorrange = cr, markersize = ms)
scatter!(OLs_ax, OLs_mark, color=:black, markersize = 10)

display(f)

GLMakie.Screen(...)